# Assignment 2: LinkedIn Webscraper

## Group A2

### Alejandra Saldivar, Anya Du Plessis, Lennart Heinacher, Henrique Villarinho & Kaustubh Singh

<div class="alert alert-success">To run this code successfully please install Selenium, BeautifulSoup4 & Numpy</div>

### Please use the following code in your terminal to install the respective packages

<b>pip install selenium</b>

<b>pip install beautifulsoup4</b>

<b>pip install numpy</b>


In [1]:
pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import requests
import bs4
import numpy as np

/Users/alejandrasaldivaraldana/opt/anaconda3/lib/python3.9/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Download [Webdriver for Chrome](https://chromedriver.chromium.org/downloads) and save the the path of the exe file in the variable **"path"** below

In [5]:
#Launch Chrome driver 
path = '/Users/Singh/Downloads/chromedriver.exe'
driver = webdriver.Chrome(path)

/var/folders/bm/9qf8nxds15zfmxyrj5d_84wm0000gn/T/ipykernel_71427/4115535945.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


<div class="alert alert-success">Web driver will launch</div>

In [6]:
# Maximize Window
driver.maximize_window() 
driver.minimize_window() 
driver.maximize_window() 
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

In [7]:
# Enter to the site
driver.get("https://www.linkedin.com/login")
time.sleep(2)

In [8]:
# Accept cookies
driver.find_element("xpath", "/html/body/div/main/div[1]/div/section/div/div[2]/button[2]").click()

You can add your own <em>User Name & Password</em> in the variable **user_name & password** below

In [9]:
# User Credentials
# Reading txt file where we have our user credentials

user_name = "" # First line
password = "" # Second line
driver.find_element("xpath", '//*[@id="username"]').send_keys(user_name)
driver.find_element("xpath", '//*[@id="password"]').send_keys(password)
time.sleep(1)

In [10]:
# Login button
driver.find_element("xpath", '//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)

In [11]:
# Access to the Jobs button and click it
driver.find_element("xpath", '//*[@id="global-nav"]/div/nav/ul/li[3]/a').click()
time.sleep(3)

# Go to search results directly via link
driver.get("https://www.linkedin.com/jobs/search/?geoId=107025191&keywords=data%20scientist&location=Barcelona%2C%20Catalonia%2C%20Spain")
time.sleep(1)

We are scrolling down the **left part** of the page while collecting the links & promoted for each job offer.

<div class="alert alert-success">Please Note: Since we are using selenium, we have to slow down the process so that account doesnt get blocked. Due to this reason, we are only retrieving 2 pages but you can increase to 40 </div>

In [12]:
# Get all links for these offers
links = []
final_url = []
promoted = []

# Navigate 2 pages
print('Links are being collected now.')
try: 
    for page in range(2,4):
        
        time.sleep(.4)
        jobs_block = jobs_block = driver.find_element(By.CSS_SELECTOR,'.scaffold-layout__list-container')
        jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
        final_url = []
        
        for job in jobs_list:
            # scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)
            time.sleep(.5)
            
            #Checking whether job is Promoted or Not
            if job.find_element("class name", 'job-card-list__footer-wrapper').text.split()[0] == "Promoted":
                promoted.append(True)
            else:
                promoted.append(False)
              
            #Using a for loop to go through all links in the job section
            all_links = job.find_elements("tag name", 'a')
            for a in all_links:
                #we are looking for attributes ‘href’ only which start with https://www.linkedin.com/jobs/view to prevent collecting unnecessary links.
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view"): 
                    final_url.append(a.get_attribute('href'))
                else:
                    pass
        
        links = links + list(set(final_url))
        
        print(f'Collecting the links in the page: {page-1}')
        
        # Go to next page:
        driver.find_element("xpath", f"//button[@aria-label='Page {page}']").click()
        time.sleep(3)      
except:
    pass

print('Found ' + str(len(links)) + ' links for job offers')

Links are being collected now.
Found 49 links for job offers


In this step we are visiting the links one by one by picking them from the list which contains all the links with the help of a for a loop.

<div class="alert alert-success">We are using both selenium and beautifulsoup to scrape data</div>

In [13]:
applicants_str = []
dates = []
details2 = []

job_titles = []
company_names = []
company_locations = []
easy_apply = []
workspace = []
state = []
type_apply = []
job_desc = []
py_val = []
num_employees = []
num_employees_int = []
num_followers = []
num_followers_int = []

python_req = ['Python', 'python']

i = 0
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')


for url in links:
    i = i + 1
    #Selenium to retrieve dynamic data
    driver.get(url)
    time.sleep(2)
    contents = driver.find_elements("class name" , 'p5')
    for content in contents:
        
        #We use try, except: pass in case there is a missing element in a page, it prevents automation from failing
        try:
        #Job titles
            job_titles.append(content.find_element("tag name","h1").text)
            
        #Company Name
            company_names.append(content.find_element("class name", "jobs-unified-top-card__company-name").text)

        #Location
            company_locations.append(content.find_element("class name", "jobs-unified-top-card__bullet").text)
   
        #Workspace
            if "Hybrid" in content.find_element("class name","jobs-unified-top-card__primary-description").text:
                workspace.append("Hybrid")
            elif "On-site" in content.find_element("class name","jobs-unified-top-card__primary-description").text:
                workspace.append("On-site")
            elif "Remote" in content.find_element("class name","jobs-unified-top-card__primary-description").text:
                workspace.append("Remote")
            else:
                workspace.append("Other")
        
        #Number of employees in the company
            num_employees.append(((content.find_element("xpath","/html/body/div[6]/div[3]/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div[2]/ul/li[2]").text).split("·")[0]).split(" ")[0])
            #num_employees_int = [int(p) for p in num_employees]
            
        #State
        #“Early Applications” if Early Applicant, "On-going" if Actively Hiring/Recruiting, or “Others” for alerts that are different

            ar = (content.find_element("class name", "mt5").text)
            if 'Early Applicant' in ar:
                state.append('Early Applications')
            elif 'Actively recruiting' in ar or 'Actively hiring' in ar:
                state.append('On-going')
            else:
                state.append('Others')
        
        #Easy Apply
            if "Easy Apply" in (content.find_element("class name","artdeco-button__text").text):
                easy_apply.append(True)
            else:
                easy_apply.append(False)
                
        except:
            break
    
    time.sleep(2)
    #Scroll down
    driver.execute_script("window.scrollTo(0, 550)")
    
    time.sleep(1)
    #Number of followers
    num_followers.append((driver.find_element("class name","artdeco-entity-lockup__content").text).split()[-2])
    #num_followers_int = [int(o) for o in num_followers]
            
    #Beautiful Soup to retrieve data
    
    #We do a get request to create a list with the section containing the information of applicants, date and python
    details_source = requests.get(url, timeout=20)
    time.sleep(2)
    details_soup = bs4.BeautifulSoup(details_source.text, 'html.parser')
        
    details_section = details_soup.find('h4',{'class':'top-card-layout__second-subline font-sans text-sm leading-open text-color-text-low-emphasis mt-0.5'})
    
    applicants_tag = details_section.find('span',{'class':'num-applicants__caption'})
    
    #Number of Applicants
    #We use an 'if' in case the job offer doesn not have the information required 
    
    if applicants_tag == None:
        if details_section.find('figcaption',{'class':'num-applicants__caption'}) == None:
            applicants_str.append(np.nan)
        else:
            applicants_str.append('25')
    else:
        applicants_str.append(applicants_tag.text.strip()[0:2])
    
    
    #Posting Date
    date_tag = details_section.find('span',{'class':'posted-time-ago__text topcard__flavor--metadata'})
    
    if date_tag == None:
        dates.append(np.nan)
    else:
        dates.append(date_tag.text.strip())     
    
    #Python Required
    
    #We retrive the section with the job offer details and look for the word "Python" or "python"
    if any(x in details_soup.find('div', {'class': 'show-more-less-html__markup show-more-less-html__markup--clamp-after-5'}).get_text(strip=True) for x in python_req):
        py_val.append(True)
    else:
        py_val.append(False)
    
    #We retrive the section with the information of seniority, employment type and industry and create a list
    section2 = details_soup.find('ul',{'class':'description__job-criteria-list'})
    
    tag = section2.find_all('span',{'class':'description__job-criteria-text description__job-criteria-text--criteria'})
    
    details2.append(tag)
    print(f"Completed scraping for {i} links")
    

Visiting the links and collecting information just started.
Completed scraping for 1 links
Completed scraping for 2 links
Completed scraping for 3 links
Completed scraping for 4 links
Completed scraping for 5 links
Completed scraping for 6 links
Completed scraping for 7 links
Completed scraping for 8 links
Completed scraping for 9 links
Completed scraping for 10 links
Completed scraping for 11 links
Completed scraping for 12 links
Completed scraping for 13 links
Completed scraping for 14 links
Completed scraping for 15 links
Completed scraping for 16 links
Completed scraping for 17 links
Completed scraping for 18 links
Completed scraping for 19 links
Completed scraping for 20 links
Completed scraping for 21 links
Completed scraping for 22 links
Completed scraping for 23 links
Completed scraping for 24 links
Completed scraping for 25 links
Completed scraping for 26 links
Completed scraping for 27 links
Completed scraping for 28 links
Completed scraping for 29 links
Completed scraping fo

As the information of seniority, employment type and industry was in one section, it was retrieved as a list of lists. In this step we are splitting it.

In [14]:
seniority = []
employment_type = []
industry = []

#Changing the 'applicants' information from strings to integers
applicants = [int(i) for i in applicants_str]
applicants

#We use a 'for loop' to split the information and create individual lists for each category
for x in range(len(details2)):
    #Seniority
    seniority.append(details2[x][0].text.strip())
    
    #Employment type
    try:
        employment_type.append(details2[x][1].text.strip())
    except IndexError:
        employment_type.append(np.nan)
    
    #Industry type
    try:
        industry.append(details2[x][3].text.strip())
    except IndexError:
        industry.append(np.nan)

In the next steps we are changing the type of the lists 'number of followers' and 'number of employees' from string form to integer form

In [15]:
num_followers_int = [int(a.replace(',', ''))for a in num_followers] 

In [16]:
num_employees_max = [d.split('-')[-0] for d in num_employees]

In [17]:
num_employees_int = [int(h.replace(',', '').replace('+',''))for h in num_employees_max]

Due to **LinkedIn bot tracker**, sometimes we are not able to retrieve the information of all the links collected. 
As we are retriving the promoted information and urls from the main page,if this happened those lists will be of different lenght. Therefore the next step is to shorten those lists.

In [18]:
#We are using an 'if' to compare the lenght of the list 'links' vs 'job_titles', in case it's different shorten them and if it's not keep them the same.
if len(links) > len(job_titles):
    link_df = []
    promoted_df = []
    for l in range(len(job_titles)):
        promoted_df.append(promoted[l])
        link_df.append(links[l])
else:
    promoted_df = promoted[:]
    link_df = links[:]

In this final step we create a dataframe with all the information retrieved.

In [19]:
import pandas as pd

#We are storing the lists that we have created into the pandas data frame
df_linkedin = pd.DataFrame(data= {'Job_Title': job_titles,
                                  'Company Name': company_names,
                                  'Location': company_locations,
                                  'State': state,
                                  'Posting Date': dates,
                                  'Offer_url': link_df,
                                  'Number of Applicants': applicants,
                                  'Promoted': promoted_df,
                                  'Workspace': workspace,
                                  'Seniority': seniority,
                                  'Employment Type': employment_type,
                                  'Industry': industry,
                                  'Python Requirement': py_val,
                                  'Easy Applicant': easy_apply,
                                  'Number of Employees': num_employees_int,
                                  'Followers': num_followers_int
                                 })

In [20]:
df_linkedin

,Job_Title,Company Name,Location,State,Posting Date,Offer_url,Number of Applicants,Promoted,Workspace,Seniority,Employment Type,Industry,Python Requirement,Easy Applicant,Number of Employees,Followers
0,Data Scientist I Candy Crush Saga,King,"Barcelona, Catalonia, Spain",On-going,NaN,https://www.linkedin.com/jobs/view/3331251159/...,62,True,On-site,Not Applicable,Full-time,Computer Games,True,False,1001,144745
1,Data Scientist - Digital Finance Hub,Novartis España,"Barcelona, Catalonia, Spain",Others,6 days ago,https://www.linkedin.com/jobs/view/3322861238/...,93,True,Other,Associate,Full-time,Pharmaceutical Manufacturing,True,False,1001,48205
2,Data Scientist,Zurich Insurance,"Barcelona, Catalonia, Spain",On-going,4 days ago,https://www.linkedin.com/jobs/view/3056003965/...,25,True,Other,Mid-Senior level,Full-time,Insurance and Financial Services,True,False,10001,695514
3,Frontend Engineer,Scandit,"Barcelona, Catalonia, Spain",On-going,2 weeks ago,https://www.linkedin.com/jobs/view/3219075642/...,25,True,Remote,Mid-Senior level,Full-time,IT Services and IT Consulting and Software Dev...,False,True,501,44128
4,Data Scientist Madrid/Barcelona/Gijón,Accenture España,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://www.linkedin.com/jobs/view/3286648063/...,14,False,Other,Mid-Senior level,Full-time,Business Consulting and Services,True,False,10001,138538
5,Senior Data Scientist,sennder,"Barcelona, Catalonia, Spain",On-going,2 weeks ago,https://www.linkedin.com/jobs/view/3191371970/...,46,False,Hybrid,Mid-Senior level,Full-time,"IT Services and IT Consulting, Software Develo...",True,False,1001,30089
6,Senior Data Scientist,AILY LABS,"Barcelona, Catalonia, Spain",Others,1 week ago,https://www.linkedin.com/jobs/view/3350985668/...,37,False,Hybrid,Full-time,NaN,NaN,True,True,51,7207
7,Data Scientist (HÍBRIDO) (a),Between Technology,"Palau-solità i Plegamans, Catalonia, Spain",On-going,NaN,https://www.linkedin.com/jobs/view/3358778960/...,26,True,Hybrid,Associate,Full-time,Software Development and Strategic Management ...,False,True,501,60352
8,Data Scientist,PepsiCo,"Barcelona, Catalonia, Spain",On-going,NaN,https://www.linkedin.com/jobs/view/3266049877/...,92,True,Hybrid,Mid-Senior level,Full-time,Food and Beverage Services and Manufacturing,True,False,10001,6879738
9,Data Scientist,PepsiCo,"Barcelona, Catalonia, Spain",On-going,1 week ago,https://www.linkedin.com/jobs/view/3343624705/...,47,False,Hybrid,Mid-Senior level,Full-time,Food and Beverage Services and Manufacturing,True,False,10001,6879739
